In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import KFold
from create_matrix import *
import os
import sys
from tensor_custom_core import *
from create_matrix import *
from tensor_custom_core import *
from degree_days import dds
from sklearn.model_selection import train_test_split, KFold
from common import compute_rmse_fraction
from common import compute_rmse

appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}
APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
year = 2014
n_splits = 10
case=2
a=2
cost='abs'

In [3]:
def un_normalize(x, maximum, minimum):
    return (maximum - minimum) * x + minimum

def get_tensor(df, dfc):
    start, stop = 1, 13
    energy_cols = np.array(
        [['%s_%d' % (appliance, month) for month in range(start, stop)] for appliance in APPLIANCES_ORDER]).flatten()

    static_cols = ['area', 'total_occupants', 'num_rooms']
    static_df = df[static_cols]
    static_df = static_df.div(static_df.max())
    weather_values = np.array(dds[2014][region][start - 1:stop - 1]).reshape(-1, 1)

    dfc = df.copy()

    df = dfc[energy_cols]
    col_max = df.max().max()
    col_min = df.min().min()
    # df = (1.0 * (df - col_min)) / (col_max - col_min)
    tensor = df.values.reshape((len(df), 7, stop - start))
    M, N, O = tensor.shape
    return tensor

In [5]:
region = "SanDiego"
sd_df, sd_dfc = create_matrix_single_region("SanDiego", year)
sd_tensor = get_tensor(sd_df, sd_dfc)
region = "Austin"
au_df, au_dfc = create_matrix_single_region("Austin", year)
au_tensor = get_tensor(au_df, au_dfc)

In [11]:
a = 2
b = 3
H_a, A_a, T_a = learn_HAT(case, au_tensor, a, b, num_iter=2000, lr=0.1, dis=False, cost_function=cost ,T_known = np.ones(12).reshape(-1, 1)) 

In [13]:
m,n,o = sd_tensor.shape

In [33]:
pred_cell = np.empty([m,n,o])

In [34]:

for i in range(m):
    for j in range(1,n):
        for k in range(o):
            print i, j, k
            tensor_copy = sd_tensor.copy()
            tensor_copy[i, j, k] = np.NaN
            
            H, A, T = learn_HAT(case, tensor_copy, a, b, num_iter=2000, lr=0.1, dis=False, cost_function=cost, A_known = A_a, T_known=np.ones(12).reshape(-1, 1))
            prediction = multiply_case(H, A, T, case)
            
            pred_cell[i][j][k] = prediction[i][j][k]

0 1 0
0 1 1
0 1 2
0 1 3
0 1 4
0 1 5
0 1 6
0 1 7
0 1 8
0 1 9
0 1 10
0 1 11
0 2 0
0 2 1
0 2 2
0 2 3
0 2 4
0 2 5
0 2 6
0 2 7
0 2 8
0 2 9
0 2 10
0 2 11
0 3 0
0 3 1
0 3 2
0 3 3
0 3 4
0 3 5
0 3 6
0 3 7
0 3 8
0 3 9
0 3 10
0 3 11
0 4 0
0 4 1
0 4 2
0 4 3
0 4 4
0 4 5
0 4 6
0 4 7
0 4 8
0 4 9
0 4 10
0 4 11
0 5 0
0 5 1
0 5 2
0 5 3
0 5 4
0 5 5
0 5 6
0 5 7
0 5 8
0 5 9
0 5 10
0 5 11
0 6 0
0 6 1
0 6 2
0 6 3
0 6 4
0 6 5
0 6 6
0 6 7
0 6 8
0 6 9
0 6 10
0 6 11
1 1 0
1 1 1
1 1 2
1 1 3
1 1 4
1 1 5
1 1 6
1 1 7
1 1 8
1 1 9
1 1 10
1 1 11
1 2 0
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
1 2 7
1 2 8
1 2 9
1 2 10
1 2 11
1 3 0
1 3 1
1 3 2
1 3 3
1 3 4
1 3 5
1 3 6
1 3 7
1 3 8
1 3 9
1 3 10
1 3 11
1 4 0
1 4 1
1 4 2
1 4 3
1 4 4
1 4 5
1 4 6
1 4 7
1 4 8
1 4 9
1 4 10
1 4 11
1 5 0
1 5 1
1 5 2
1 5 3
1 5 4
1 5 5
1 5 6
1 5 7
1 5 8
1 5 9
1 5 10
1 5 11
1 6 0
1 6 1
1 6 2
1 6 3
1 6 4
1 6 5
1 6 6
1 6 7
1 6 8
1 6 9
1 6 10
1 6 11
2 1 0
2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 1 8
2 1 9
2 1 10
2 1 11
2 2 0
2 2 1
2 2 2
2 2 3
2 2 4
2 2 5
2 

In [59]:
from scipy.optimize import nnls

n_splits = 10
case = 2
a = 2
b = 3
cost = 'abs'


pred_cv = {}
for random_seed in range(10):
    
    pred_cv[random_seed] = {}
    for appliance in APPLIANCES_ORDER:
        pred_cv[random_seed][appliance] = {f:[] for f in range(10, 110, 10)}

kf = KFold(n_splits=n_splits)
for random_seed in range(5):
    print "random seed: ", random_seed
    for train_percentage in range(10, 110, 10):
        print "training percentage: ", train_percentage
        rd = 0
        for train_max, test in kf.split(sd_df):
                print "round: ", rd

                num_train = int((train_percentage*len(train_max)/100)+0.5)
                num_test = len(test)

                # get the random training data from train_max based on then random seed
                if train_percentage==100:
                    train = train_max
                else:
                    train, _ = train_test_split(train_max, train_size = train_percentage/100.0, random_state=random_seed)

                # get the index of training and testing data
                train_ix = sd_df.index[train]
                test_ix = sd_df.index[test]
                print "test_ix: ", test_ix

                # create the tensor
                train_test_ix = np.concatenate([test_ix, train_ix])
                df_t, dfc_t = sd_df.ix[train_test_ix], sd_dfc.ix[train_test_ix]
                tensor = get_tensor(df_t, dfc_t)


                #############################################################################################
                # transfer learning: constant constraint, with A_known = A_a_const, with learn_HAT_constant
                ############################################################################################
                pred = np.empty([num_test, 7, 12])
                
                for i in range(1, 7):
                    
                    tensor_copy = tensor.copy()
                    tensor_copy[:num_test, i, :] = np.NaN
                    H, A, T = learn_HAT(case, tensor_copy, a, b, num_iter=2000, lr=0.1, dis=False, cost_function=cost, A_known=A_a,T_known = np.ones(12).reshape(-1, 1))
                    HAT = multiply_case(H, A, T, case)
                    pred[:, i, :] = HAT[:num_test, i, :]
                # get the prediction
                
                for appliance in APPLIANCES_ORDER:
                    pred_cv[random_seed][appliance][train_percentage].append(pd.DataFrame(pred[:, appliance_index[appliance], :], index=test_ix))

                rd += 1

random seed:  0
training percentage:  10
round:  0
test_ix:  Int64Index([54, 203, 527, 1450], dtype='int64')
round:  1
test_ix:  Int64Index([1524, 1731, 2031, 2354], dtype='int64')
round:  2
test_ix:  Int64Index([2606, 3687, 3864, 3938], dtype='int64')
round:  3
test_ix:  Int64Index([4083, 4095, 4329, 4495], dtype='int64')
round:  4
test_ix:  Int64Index([4761, 4934, 5909, 5938], dtype='int64')
round:  5
test_ix:  Int64Index([6268, 6377, 6429, 6497], dtype='int64')
round:  6
test_ix:  Int64Index([6547, 7062, 7114, 7409], dtype='int64')
round:  7
test_ix:  Int64Index([7639, 8061, 8342, 8574], dtype='int64')


KeyboardInterrupt: 

In [54]:
pred_cv[0]['dw'][10]

[             0          1          2          3          4          5   \
 54     4.441050   4.374520   4.457078   4.687009   4.945526   5.192324   
 203    5.503513   5.430953   5.521882   5.773382   6.090897   6.353401   
 527   11.354342  11.216256  11.390482  11.870073  12.521796  13.012461   
 1450  11.623782  11.452640  11.665282  12.256978  12.932745  13.565594   
 
              6          7          8          9          10         11  
 54     4.288658   5.120256   5.142503   5.153532   4.816868   5.673344  
 203    5.416679   6.326773   6.352496   6.309071   5.915830   6.851801  
 527   11.295028  13.031156  13.082057  12.925400  12.142253  13.925949  
 1450  11.255776  13.395926  13.453595  13.465194  12.591277  14.795044  ,
              0          1          2          3          4          5   \
 1524  10.809674  10.263070  10.387690  11.045619  11.730628  12.102732   
 1731   9.438079   8.860635   8.989631   9.682273  10.311600  10.761934   
 2031   4.138151   3.872244

In [38]:
pred_cell[:, 1, :]

array([[   0.        ,    0.        ,    0.        ,   11.18723443,
          85.89969937,   54.95676031,  138.22499142,  165.33004297,
         174.06390724,   64.82408541,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,   16.36920788,
         122.07919527,   79.19176856,  196.30009438,  243.430618  ,
         282.42734073,   99.45670315,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,   39.09422878,
         291.55912891,  194.69024705,  496.86355717,  573.53475427,
         598.77516629,  228.69533   ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,   34.03037097,
         227.21695396,  149.7512468 ,  391.03006304,  424.19545117,
         456.80554756,  163.1795575 ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,   40.52508894,
         302.23027798,  201.81596682,  515.04890814,  622.49467606,
         683.39460301,  253.12000959,    0. 

In [41]:
out = {}
for appliance in APPLIANCES_ORDER[1:]:
    out[appliance] = {}
    
    s = pd.DataFrame(pred_cell[:, appliance_index[appliance], :], index = sd_df.index)
    
    if appliance=="hvac":
        out[appliance] = compute_rmse_fraction(appliance,s[range(4, 10)],'SanDiego')[2]
    else:   
        out[appliance] = compute_rmse_fraction(appliance, s,'SanDiego')[2]

In [42]:
out

{'dw': 1.8419311486511902,
 'fridge': 9.8563926947604124,
 'hvac': 13.355191358765531,
 'mw': 2.0429952976254757,
 'oven': 2.0290824167898225,
 'wm': 0.80628242095805347}

In [55]:
out_cv = {}
for random_seed in range(1):
    out_cv[random_seed] = {}
       
    for appliance in APPLIANCES_ORDER[1:]:
        out_cv[random_seed][appliance] = {}
        for f in range(10,110,10):
            s = pd.concat(pred_cv[random_seed][appliance][f]).ix[sd_df.index]
            if appliance=="hvac":
                out_cv[random_seed][appliance][f] = compute_rmse_fraction(appliance,s[range(4, 10)],'SanDiego')[2]
            else:   
                out_cv[random_seed][appliance][f] = compute_rmse_fraction(appliance, s,'SanDiego')[2]

/home/yj9xs/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [56]:
out_cv

{0: {'dw': {10: 1.8031070611039735,
   20: 1.8326450208294032,
   30: 1.8038026265539313,
   40: 1.8562518352890098,
   50: 1.8744035228799558,
   60: 1.8483923267952369,
   70: 1.8507870807468458,
   80: 1.842594491431595,
   90: 1.8420906961574237,
   100: 1.8428020258278546},
  'fridge': {10: 10.701905435848772,
   20: 10.527938352621847,
   30: 10.684484473954527,
   40: 10.411748350813509,
   50: 10.17080369963093,
   60: 10.209669109504148,
   70: 10.121152718422511,
   80: 10.188205732798044,
   90: 10.126104985479644,
   100: 10.091692686517421},
  'hvac': {10: 20.247335912746109,
   20: 17.806249405158781,
   30: 16.958587764409074,
   40: 18.045308921281126,
   50: 17.58932289709664,
   60: 16.7428410729934,
   70: 17.158650676407994,
   80: 17.806860893640728,
   90: 17.24657635798247,
   100: 17.114123638732384},
  'mw': {10: 1.9471950448218962,
   20: 1.9558842773138152,
   30: 1.9971757148721281,
   40: 1.9929438435113103,
   50: 1.9806324741364991,
   60: 2.0133654358642